原文连接：[Using pre-trained word embeddings in a Keras model](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

中文文档：[在Keras模型中使用预训练的词向量](http://keras-cn-docs.readthedocs.io/zh_CN/latest/blog/word_embedding/)

本文将讲述使用预训练的词向量和卷积神经网络来实现文本分类问题。

### 实验方法
以下是我们如何解决分类问题的步骤

- 将所有的新闻样本转化为词索引序列。所谓词索引就是为每一个词依次分配一个整数ID。遍历所有的新闻文本，我们只保留最参见的20,000个词，而且 每个新闻文本最多保留1000个词。
- 生成一个词向量矩阵。第i列表示词索引为i的词的词向量。
- 将词向量矩阵载入Keras Embedding层，设置该层的权重不可再训练（也就是说在之后的网络训练过程中，词向量不再改变）。
- Keras Embedding层之后连接一个1D的卷积层，并用一个softmax全连接输出新闻类别

### 数据预处理


In [2]:
import os
import sys

查看训练样本文件以及内容

In [3]:
fpath1 = '/home/panxie/Documents/cs231n/myNotes_rnn/keras_models/20_newsgroup/alt.atheism/49960'
with open(fpath1, encoding='latin-1') as f:
    t = f.read()
    print("################")
    print(t[:1000])
    i = t.find('\n\n')
    print("################")
    print(i)  ##904 意味着前面905个字符删掉，删除头部信息，保留新闻信息
    if i > 0:
        t = t[i:]
    print(t[:100])

################
Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew
From: mathew <mathew@mantis.co.uk>
Newsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers
Subject: Alt.Atheism FAQ: Atheist Resources
Summary: Books, addresses, music -- anything related to atheism
Keywords: FAQ, atheism, books, music, fiction, addresses, contacts
Message-ID: <19930329115719@mantis.co.uk>
Date: Mon, 29 Mar 1993 11:57:19 GMT
Expires: Thu, 29 Apr 1993 11:57:19 GMT
Followup-To: alt.atheism
Distribution: world
Organization: Mantis Consultants, Cambridge. UK.
Approved: news-answers-request@mit.edu
Supersedes: <19930301143317@mantis.co.uk>
Lines: 290

Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modi

遍历训练数据下的文件夹，并获得不同类别的新闻以及对应的类别标签.
每一个文件即新闻作为一个sequence，并存放在texts列表中

In [4]:
### 遍历训练数据下的文件夹，并获得不同类别的新闻以及对应的类别标签，
texts = []
labels_index = {}
labels = []
TEXT_DATA_DIR = '/home/panxie/Documents/cs231n/myNotes_rnn/keras_models/20_newsgroup'
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index) ## 类别标签从０开始加１
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # 把每个文本文件中头部信息，非新闻信息删掉了。
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 19997 texts.


对样本数据texts进行词分割放入列表sequences中，并保留其中最常见的20000个词。即sequences中每个元素是一个新闻分词后的列表。

In [5]:
###把文本训练样本数据和标签转换为词向量矩阵。(19997,1)->(50, 19997)
MAX_NB_WORDS = 20000

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 类实例化
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) ##num_words保留最常见的词数量，这里是20000
# texts要用以训练的文本列表,喂入文本数据
tokenizer.fit_on_texts(texts)
# 
sequences = tokenizer.texts_to_sequences(texts) #返回：序列的列表，列表中每个序列对应于一段输入文本
print(len(sequences[0])) ## 第一个新闻有多少个词
print(len(sequences))  ## 19997个样本sequences
print(sequences[0][-10:]) # 将文本中每个词转换为其对应在字典中的索引

Using TensorFlow backend.


1528
19997
[1213, 2632, 5, 11, 41, 176, 173, 4, 930, 2050]


我们用另一个类方法句子分割，来对比看一下第一个新闻中词的数量，发现直接句子分割要上面的方法要多，这是因为不常见的被去掉了

In [6]:
from keras.preprocessing.text import text_to_word_sequence
sequences_0 = text_to_word_sequence(texts[0]) ## 句子分割
print(len(sequences_0))
print(sequences_0[:10])

1733
['archive', 'name', 'atheism', 'resources', 'alt', 'atheism', 'archive', 'name', 'resources', 'last']


我们发现name对应273, archive对应1237.　为什么1733>1528，因为只保留了最常见的20000个词，故对于第一个句子，有部分词被删除了吧。

tokenizer的一个属性word_index，用来查看训练样本每个词以及他们的索引

In [7]:
### hash table
### word_index是训练样本中每个词以及其对应的索引。
word_index = tokenizer.word_index  ### 字典，将单词（字符串）映射为它们的排名或者索引。仅在调用fit_on_texts之后设置。
print("Found %s unique tokens." % len(word_index))
print(word_index['name'])

Found 174074 unique tokens.
273


In [8]:
MAX_SEQUENCE_LENGTH = 2000 ## 每篇新闻只取前2000个词
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) ##　不足2000词的padding

print(data.shape)
print(data[0,:]) # 从开头padding的啊

(19997, 2000)
[   0    0    0 ...,    4  930 2050]


In [9]:
from keras.utils import to_categorical
import numpy as np

print(len(labels))
labels = to_categorical(np.asarray(labels))  ##　总共有20类，通过计算可得到。
print("shape of data tensor:", data.shape)
print("shape of label tensor:", labels.shape)

19997
shape of data tensor: (19997, 2000)
shape of label tensor: (19997, 20)


In [10]:
# split the data into a training set and a validation set
VALIDATION_SPILT = 0.01
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPILT * data.shape[0]) ## 0.01

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [11]:
print(y_train.shape)
print(y_val.shape)

(19798, 20)
(199, 20)


### preparing the Embedding layer
接下来，我们从GloVe文件中解析出每个词和它所对应的词向量，并用字典的方式存储

In [12]:
### 从50d的glove词向量获取词向量map
embeddings_index = {}
GLOVE_DIR = '/home/panxie/Documents/cs231n/myNotes_rnn/keras_models'
with open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index)) 
print(embeddings_index['apple'])

Found 400000 word vectors.
[ 0.52042001 -0.83139998  0.49961001  1.28929996  0.1151      0.057521
 -1.37530005 -0.97312999  0.18346     0.47672001 -0.15112001  0.35532001
  0.25911999 -0.77857     0.52181     0.47694999 -1.42509997  0.85799998
  0.59820998 -1.09029996  0.33574    -0.60891002  0.41742     0.21569
 -0.07417    -0.58219999 -0.45019999  0.17253     0.16448    -0.38413
  2.3283     -0.66681999 -0.58181     0.74388999  0.095015   -0.47865
 -0.84591001  0.38703999  0.23693    -1.55229998  0.64802003 -0.16520999
 -1.47189999 -0.16224     0.79856998  0.97390997  0.40026999 -0.21912
 -0.30937999  0.26581001]


In [13]:
## 根据得到的字典生成上文所定义的词向量矩阵
EMBEDDING_DIM = embeddings_index['apple'].shape[0]
embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word) #将word_index中每个词换成其在GLOVE词向量中对应的向量
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector ##嵌入到训练样本对应的词向量中去
print(embedding_matrix.shape) ## 174075 < 19997　说明有些词在GLOVE里面没有。。

(174075, 50)


### Embedding layer

In [14]:
from keras.layers import Embedding

embedding_layer = Embedding(input_dim=len(word_index)+1, output_dim=EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable=False)
##对于Embedding层
## input: (174075, 2000)
## output: (174075, 2000, 50)

一个Embedding层的输入应该是一系列的整数序列，比如一个2D的输入，它的shape值为(samples, indices)，也就是一个samples行，indeces列的矩阵。每一次的batch训练的输入应该被padded成相同大小（尽管Embedding层有能力处理不定长序列，如果你不指定数列长度这一参数） dim). 所有的序列中的整数都将被对应的词向量矩阵中对应的列（也就是它的词向量）代替,比如序列[1,2]将被序列[词向量[1],词向量[2]]代替。这样，输入一个2D张量后，我们可以得到一个3D张量(samples, sequence_length, embedding_dim).。

### Training a 1D convnet

In [16]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') #(2000,None)
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)  ## global max pooling
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train,y_train,
         batch_size=128,
         epochs=10,
         validation_data=(x_val, y_val))

Train on 19798 samples, validate on 199 samples
Epoch 1/10
19798/19798 [==============================] - 266s 13ms/step - loss: 2.2871 - acc: 0.2398 - val_loss: 1.8404 - val_acc: 0.3618
Epoch 2/10
19798/19798 [==============================] - 246s 12ms/step - loss: 1.5756 - acc: 0.4495 - val_loss: 1.6407 - val_acc: 0.4925
Epoch 3/10
19798/19798 [==============================] - 270s 14ms/step - loss: 1.2733 - acc: 0.5606 - val_loss: 1.3420 - val_acc: 0.5528
Epoch 4/10
19798/19798 [==============================] - 276s 14ms/step - loss: 1.0905 - acc: 0.6289 - val_loss: 1.0659 - val_acc: 0.6683
Epoch 5/10
19798/19798 [==============================] - 264s 13ms/step - loss: 0.9793 - acc: 0.6727 - val_loss: 1.1185 - val_acc: 0.6533
Epoch 6/10
19798/19798 [==============================] - 239s 12ms/step - loss: 0.8754 - acc: 0.7057 - val_loss: 1.1205 - val_acc: 0.6784
Epoch 7/10
19798/19798 [==============================] - 240s 12ms/step - loss: 0.7963 - acc: 0.7308 - val_loss: 1.08